In [19]:
%pip install google-genai
%pip install python-dotenv
%pip install beautifulsoup4
%pip install googlesearch-python


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from google import genai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
cx_id = os.getenv("GOOGLE_CSE_ID")
client = genai.Client(api_key=api_key)

eTake the input from file 'data/generated_recipes.json' and check each recipe for correctness.

The RAG model first takes the name of the medicine from field 'medicijn' and retrieves the documentation from apotheek.nl. The pattern for the URL is:
https://www.apotheek.nl/medicijnen/{medicijn}

Then it checks if the medicine is valid by checking if the URL returns a 200 status code. If the medicine can not be found, the ai model is prompted to provide a single url to the documentation of the medicine on apotheek.nl. Then again the URL is checked for a 200 status code. If this still fails, the recipe is considered invalid.

If the medicine is valid the documentation is provided as context to the AI model to check if the strength, form, number of doses, and instructions are valid. The AI model is also asked to check if the indication is relevant to the medical question.


In [2]:
from bs4 import BeautifulSoup
import unicodedata
import requests

def read_medicijn_details(html_text: str):
    """Extracts the medicine details from the HTML text."""
    soup = BeautifulSoup(html_text, 'html.parser')
    ul_element = soup.find('ul', id='container')
    if ul_element:
        return ul_element.get_text(strip=True)
    else:
        return "Geen details gevonden voor het medicijn."

def get_medicijn_details(medicijn: str):
    # use google search to find the medicine documentation on apotheek.nl
    query = f"{medicijn} site:apotheek.nl"
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cx_id}"
    search_response = requests.get(url).json()

    if 'items' in search_response and len(search_response['items']) > 0:
        medicijn_url = search_response['items'][0]['link']
        return read_medicijn_details(requests.get(medicijn_url).text)
    else:
        return "Geen details gevonden voor het medicijn."

print(get_medicijn_details("Triamcinolonacetonide"))

Belangrijk om te weten over triamcinolonacetonide op de huidTriamcinolonacetonide is een bijnierschorshormoon (corticosteroïd). Het remt ontstekingen en maakt schilfers, jeuk en zwellingen minder.Bij huidaandoeningen met ontstekingen. Zoals eczeem, seborroïsch eczeem, jeuk, psoriasis en lichtovergevoeligheid.U heeft binnen een paar uur minder jeuk.Uw klachten als roodheid en schilfers worden binnen een paar dagen minder.Dit medicijn kan de verschijnselen van een huidinfectie maskeren. U merkt dus minder snel dat u een infectie op uw huid heeft. Dit komt doordat dit medicijn de verschijnselen van een infectie remt, zoals jeuk, zwellingen en roodheid. De infectie kan zich daardoor uitbreiden zonder dat u het merkt.Heeft u een infectie op uw huid? Of denkt u dat u een infectie op uw huid heeft? Gebruik dit medicijn niet op deze plek. Gebruik dit medicijn bijvoorbeeld niet op of in de buurt van voetschimmel, zweren, gordelroos en een koortslip. Gebruikt u ook een medicijn tegen deze infect

Now use the medical docs to check each recipe for correctness.


In [21]:
def check_recipe_rag(recipe, details):
    """Check the recipe using the RAG model."""
    medical_question = recipe.get('medische_vraag')
    medicijn = recipe.get('medicijn')
    recept = recipe.get('recept')

    prompt = f"""
    Controleer het volgende recept op juistheid:

    Vraag: {medical_question}

    Recept: {recept}

    Details: {details}
    """

    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            "response_schema": {
                "type": "object",
                "properties": {
                    "correct": {"type": "boolean"},
                    "explanation": {"type": "string"}
                },
                "required": ["correct", "explanation"]
            },
            "system_instruction": ("Beoordeel met de tekst in 'Details' het recept op basis van de volgende criteria: "
                                  "1. Bevat het medicijn een geldige naam, sterkte, vorm, aantal doses en instructies? "
                                  "2. Is de indicatie relevant voor de medische vraag? "
                                  "3. Geef een beoordeling van het recept als correct = True|False. ")
        }
    )
    return json.loads(response.text)



In [24]:
import json

# Read the JSON file
with open('../data/generated_recipes.json', 'r', encoding='utf-8') as file:
    recipes = json.load(file)

# Loop through each recipe and access the 'medicijn' field
for recipe in recipes:
    medicijn = recipe.get('medicijn')
    details = get_medicijn_details(medicijn)
    if details.startswith('Geen details gevonden'):
        recipe['correct'] = False
        recipe['explanation'] = "Medicijn niet gevonden op apotheek.nl."
    else:
        # prompt the AI model to check the recipe
        # add the documentation to the prompt
        result = check_recipe_rag(recipe, details)
        recipe['correct'] = result['correct']
        recipe['explanation'] = result['explanation']

# Write the recipes to a JSON file
output_file = "../data/generated_recipes_rag_review.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(recipes, f, ensure_ascii=False, indent=4)

